# Installing Hub

In [ ]:
!pip3 install hub --quiet

# Run below cells and restart the runtime
# if you are running it in colab
# import os
# os.kill(os.getpid(), 9) 

# Download raw dataset

In [ ]:
from IPython.display import clear_output
from google.colab import files

In [ ]:
#@title Login to Kaggle
files.upload() 
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print('Kaggle Json Uploaded')

In [ ]:
# Download dataset here
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
!mkdir kaggle/
!mv *.csv *.gz *.zip kaggle/

# Unzip files
!unzip kaggle/icml_face_data.csv.zip
!tar -xvzf kaggle/fer2013.tar.gz
!unzip kaggle/test.csv.zip 
!unzip kaggle/train.csv.zip

clear_output()

In [ ]:
import numpy as np 
from matplotlib import pyplot as plt

# Creating dataset on hub

**Activeloop API** : https://docs.activeloop.ai/api-basics

In [ ]:
import hub

# Login to ActiveLoop

!activeloop login -u username -p password
!activeloop reporting --off

In [ ]:
!rm -rf FER*

In [ ]:
filename_path = 'hub://<username>/fer2013'
train_ds = hub.dataset(filename_path+'-train')
public_test_ds = hub.dataset(filename_path+'-public-test')
private_test_ds = hub.dataset(filename_path+'-private-test')

In [ ]:
data = pd.read_csv('icml_face_data.csv')
print(f"Columns : {data.columns}")
data.groupby(by=' Usage').count()

In [ ]:
# 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

In [ ]:
with train_ds:
    train_ds.info.update(source='https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/',
                   title='Challenges in Representation Learning: Facial Expression Recognition Challenge',
                   classes='0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral')
    
    # Training data
    train_ds.create_tensor('images', htype='image', sample_compression=None)
    train_ds.create_tensor('labels', htype = 'class_label', class_names = class_names)

with public_test_ds:
    public_test_ds.info.update(source='https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/',
                title='Challenges in Representation Learning: Facial Expression Recognition Challenge',
                classes='0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral')

    # Testing data
    public_test_ds.create_tensor('images', htype='image', sample_compression=None)
    public_test_ds.create_tensor('labels', htype = 'class_label', class_names = class_names)

with private_test_ds:
    private_test_ds.info.update(source='https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/',
                title='Challenges in Representation Learning: Facial Expression Recognition Challenge',
                classes='0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral')

    # Testing data
    private_test_ds.create_tensor('images', htype='image', sample_compression=None)
    private_test_ds.create_tensor('labels', htype = 'class_label', class_names = class_names)

In [ ]:
def convert_string_2_matrix(string):
    array = [int(data) for data in string.split(' ')]
    matrix = np.array(array, dtype='uint8').reshape((48, 48))
    return matrix

In [ ]:
for group_name, df_group in data.groupby(by=' Usage'):
    if group_name == 'Training':
        with train_ds:
            for index, row in df_group.iterrows():
                matrix = convert_string_2_matrix(row[' pixels'])
                train_ds['images'].append(matrix)
                train_ds['labels'].append(np.uint32(row['emotion']))

    if group_name == 'PublicTest':
        with public_test_ds:
            for index, row in df_group.iterrows():
                matrix = convert_string_2_matrix(row[' pixels'])
                public_test_ds['images'].append(matrix)
                public_test_ds['labels'].append(np.uint32(row['emotion']))

    if group_name == 'PrivateTest':
        with private_test_ds:
            for index, row in df_group.iterrows():
                matrix = convert_string_2_matrix(row[' pixels'])
                private_test_ds['images'].append(matrix)
                private_test_ds['labels'].append(np.uint32(row['emotion']))

In [ ]:
image = train_ds['images'][0].numpy()
label = train_ds['labels'][0].numpy()[0]
print(f'Train Label -> {label}')
plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
image = public_test_ds['images'][0].numpy()
label = public_test_ds['labels'][0].numpy()[0]
print(f'Public Test Label -> {label}')
plt.imshow(image, cmap='gray')
plt.show()

In [ ]:
image = private_test_ds['images'][0].numpy()
label = private_test_ds['labels'][0].numpy()[0]
print(f'Private Test Label -> {label}')
plt.imshow(image, cmap='gray')
plt.show()